In [2]:
%matplotlib qt

In [1]:
import os, glob, platform
import numpy as np
import numpy.matlib
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# mne libraries
import shutil as sh
import mne


In [ ]:
#%% FUNCTIONS
def len2(x):
	if type(x) is not type([]):
		if type(x) is not type(array([])):
			return -1
	return len(x)

def circdist(angles1,angles2):
    if len2(angles2) < 0:
        if len2(angles1) > 0:
            angles2 = [angles2]*len(angles1)
        else:
            angles2 = [angles2]
            angles1 = [angles1]
    if len2(angles1) < 0:
        angles1 = [angles1]*len(angles2)
        
    return np.angle(exp(1j*angles1)/exp(1j*angles2))
        
    #return amap(lambda a1,a2: phase2(exp(1j*a1)/exp(1j*a2)), angles1,angles2)

def circ_mean(x):
	return circmean(x,low=-pi,high=pi)

def rem_sys_err(report,target,w):
    n=[]
    for x in arange(-pi,pi,w):
        d=circdist(target,x)
        idx = (d >= x) & (d<x+w)
        report[idx] = circdist(report[idx],circmean(circdist(report[idx],target[idx]),low=-pi,high=pi))
        n.append(sum(idx))
        
    return report,n

def rem_sys_err2(report,target,frac=.25):
    #''' Removes from the response the systematic error of the subject.'''
    fit_resp = loess.lowess(report, target, frac = frac, return_sorted = False)
    fit_error = circdist(fit_resp, target)
    clean_res = circdist(report, fit_error) # Clean response ?
    return clean_res

def compute_serial(report,target,pre_target,xxx,flip=None,ref_report=False):
	report = rem_sys_err2(array(report),array(target))
	err = circdist(report,target)
	d = circdist(pre_target,target) #Previous cue - current cue
	if ref_report:
		d=circdist(report[:-1],target[1:])
	
	# remove outliers
	idx_clean=(abs(err)<3*std((err))) & (abs(err)<1)
	#idx_clean = idx_clean & (report[1:] !=0)
	
	err=err[idx_clean]
	d=d[idx_clean]
	m_err=[]
	std_err=[]
	count=[]
	cis=[]
	if flip:
		err = sign(d)*err
		d=abs(d)
	points_idx=[]
	for t in xxx:
		idx=(d>=t)&(d<=t+w2)
		m_err.append(circ_mean(err[idx]))
		std_err.append(circstd(err[idx])/sqrt(sum(idx)))
		count.append(sum(idx))
		points_idx.append(idx)
	return array(err),d,array(m_err),array(std_err),count,points_idx


__Participant and session characteristics__

In [3]:
# Importing behavioral data from experiment 
if platform.system() == 'Darwin':
    mne_dir = os.path.join('/Users','alex','Library','CloudStorage','OneDrive-UniversitatdeBarcelona','Projects','IDIBAPS','Exp_3_WM_prior') # mac
if platform.system() == 'Linux':
     mne_dir = os.path.join('/home','freeman','Insync', 'alexisperez@ub.edu', 'OneDrive Biz', 'PROJECTS','IDIBAPS','Exp_3_WM_prior')  # Linux 

In [5]:
#data_dir = os.path.join(mne_dir , 'behav')
#print(data_dir)


/Users/alex/Library/CloudStorage/OneDrive-UniversitatdeBarcelona/Projects/IDIBAPS/Exp_3_WM_prior/behav


loading data

Checking data crop

__Loading behavioral data__

In [14]:
results_path =  os.path.join(mne_dir,'Behav','pilot') 

print(results_path)
subj_sel = 'Leen'

# loading and cocatenating data
os.chdir(results_path)
pd.DataFrame([])
os.chdir(results_path)
bdata = pd.DataFrame([])

for file in glob.glob("*.csv"):
    print(file)
    bdata = bdata.append(pd.read_csv(os.path.join(results_path, file),sep=';'))
    #dat = pickle.load(p_file)


print(results_path)

#df = pd.concat([rdata, ddata, odata], axis = 1)


/Users/alex/Library/CloudStorage/OneDrive-UniversitatdeBarcelona/Projects/IDIBAPS/Exp_3_WM_prior/Behav/pilot
Leen_020220232104.csv
/Users/alex/Library/CloudStorage/OneDrive-UniversitatdeBarcelona/Projects/IDIBAPS/Exp_3_WM_prior/Behav/pilot


In [15]:
bdata

,subj,trial,block,prior,ct,delay,fixated,keypressed,R,T_Angle,...,MT,ts_b,ts_f,ts_p,ts_d,ts_r,ts_e,m_pos_x,m_pos_y,m_clock
0,Leen,0,0,0,False,3000,1,NaN,10,-113.216174,...,1.950312,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,"[-52.883606, -52.883606, -52.883606, -52.88360...","[-43.268406, -43.268406, -43.268406, -43.26840...","[0.001176932011730969, 0.03317452099872753, 0...."
1,Leen,1,0,0,False,1000,1,NaN,10,-34.676601,...,2.099938,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,"[-57.88098, -57.88098, -57.88098, -57.88098, -...","[-42.066505, -42.066505, -42.066505, -42.06650...","[0.001642660005018115, 0.01693040900863707, 0...."
2,Leen,2,0,0,False,3000,1,NaN,10,102.311261,...,4.450459,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,"[-44.59682, -44.59682, -44.59682, -44.59682, -...","[-28.149767, -28.149767, -28.149767, -28.14976...","[0.0014265609788708389, 0.01646969700232148, 0..."
3,Leen,3,0,0,False,1000,1,NaN,10,20.665683,...,2.949615,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,"[-46.81085, -46.81085, -46.81085, -46.81085, -...","[-40.991123, -40.991123, -40.991123, -40.99112...","[0.0016278569819405675, 0.015970641979947686, ..."
4,Leen,4,0,0,False,1000,1,NaN,10,177.726916,...,1.933836,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,1.675368e+09,"[-58.323788, -58.323788, -58.323788, -58.32378...","[-41.813477, -41.813477, -41.813477, -41.81347...","[0.00140902103157714, 0.016790005029179156, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Leen,45,9,[192.98077725],False,1000,1,NaN,10,302.299494,...,1.400266,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,"[-44.343792, -44.343792, -44.343792, -44.34379...","[-28.529314, -28.529314, -28.529314, -28.52931...","[0.0018305809935554862, 0.01702922600088641, 0..."
496,Leen,46,9,[192.98077725],False,3000,1,NaN,10,135.240158,...,1.767859,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,"[-58.450302, -58.450302, -58.450302, -58.45030...","[-24.733839, -24.733839, -24.733839, -24.73383...","[0.0011958259856328368, 0.016467955021653324, ..."
497,Leen,47,9,[192.98077725],False,1000,1,NaN,10,359.187367,...,2.348710,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,"[-52.693832, -52.693832, -52.693832, -52.69383...","[-43.205147, -43.205147, -43.205147, -43.20514...","[0.0018478929996490479, 0.01670226000715047, 0..."
498,Leen,48,9,[192.98077725],False,3000,1,NaN,10,216.134817,...,1.883755,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,1.675371e+09,"[-50.796093, -50.796093, -50.796093, -50.79609...","[-42.952114, -42.952114, -42.952114, -42.95211...","[0.0018594390130601823, 0.015783348004333675, ..."
